# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up GraphLab Create

Make sure you have the latest version of GraphLab Create.

In [1]:
from __future__ import division
import graphlab
import math
import string

# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
products = graphlab.SFrame('amazon_baby.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to arslanahmadid@gmail.com and will expire on October 28, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.0.1 started. Logging: C:\Users\arslan\AppData\Local\Temp\graphlab_server_1551519673.log.0


Now, let us see a preview of what the dataset looks like.

In [3]:
print products

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|    Planetwise Flannel Wipes   | These flannel wipes are OK... |  3.0   |
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
+------------------------

## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [4]:
products[269]

{'name': 'The First Years Massaging Action Teether',
 'rating': 5.0,
 'review': 'A favorite in our house!'}

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [5]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

review_without_punctuation = products['review'].apply(remove_punctuation)
products['word_count'] = graphlab.text_analytics.count_words(review_without_punctuation)

In [6]:
products['review'][0]

'These flannel wipes are OK, but in my opinion not worth keeping.  I also ordered someImse Vimse Cloth Wipes-Ocean Blue-12 countwhich are larger, had a nicer, softer texture and just seemed higher quality.  I use cloth wipes for hands and faces and have been usingThirsties 6 Pack Fab Wipes, Boyfor about 8 months now and need to replace them because they are starting to get rough and have had stink issues for a while that stripping no longer handles.'

In [7]:
review_without_punctuation[0]

'These flannel wipes are OK but in my opinion not worth keeping  I also ordered someImse Vimse Cloth WipesOcean Blue12 countwhich are larger had a nicer softer texture and just seemed higher quality  I use cloth wipes for hands and faces and have been usingThirsties 6 Pack Fab Wipes Boyfor about 8 months now and need to replace them because they are starting to get rough and have had stink issues for a while that stripping no longer handles'

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [8]:
products[269]['word_count']

{'a': 1L, 'favorite': 1L, 'house': 1L, 'in': 1L, 'our': 1L}

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [9]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [10]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
print products

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
| Baby Tracker&reg; - Daily ... | I love this journal and ou... |  4.0   |
+------------------------

Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [11]:
train_data, test_data = products.random_split(.8, seed=1)
print len(train_data)
print len(test_data)

133416
33336


# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [12]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                      target = 'sentiment',
                                                      features=['word_count'],
                                                      validation_set=None)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 12       | 3.000000  | 6.415571     | 0.960964          |

| 6         | 13       | 3.000000  | 6.993271     | 0.975033          |

+-----------+----------+-----------+--------------+-------------------+

TERMINATED: Terminated due to numerical difficulties.

This model may not be ideal. To improve it, consider doing one of the following:
(a) Increasing the regularization.
(b) Standardizing the input data.
(c) Removing highly correlated features.
(d) Removing `inf` and `NaN` values in the training data.

In [13]:
sentiment_model

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 121713
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 121712

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : lbfgs
Solver iterations              : 6
Solver status                  : TERMINATED: Terminated due to numerical difficulties.
Training time (sec)            : 7.553

Settings
--------
Log-likelihood                 : inf

Highest Positive Coefficients
-----------------------------
word_count[mobileupdate]       : 41.9847
word_count[placeid]            : 41.7354
word_count[labelbox]           : 41.151
word_count[httpwwwamazoncomreviewrhgg6qp7tdnhbrefcmcrprcmtieutf8asinb00318cla0nodeid] : 40.0454
word_count[knobskeeping]       : 36.2091

Lowest Negative Coeffic

**Aside**. You may get a warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [14]:
weights = sentiment_model.coefficients
print weights
print weights.column_names()


+-------------+--------------+-------+------------------+--------+
|     name    |    index     | class |      value       | stderr |
+-------------+--------------+-------+------------------+--------+
| (intercept) |     None     |   1   |  1.30337080544   |  None  |
|  word_count |  recommend   |   1   |  0.303815600015  |  None  |
|  word_count |    highly    |   1   |  1.49183015276   |  None  |
|  word_count | disappointed |   1   |  -3.95748618393  |  None  |
|  word_count |     love     |   1   |  1.43301685439   |  None  |
|  word_count |      it      |   1   | 0.00986646490307 |  None  |
|  word_count |    planet    |   1   | -0.797764553926  |  None  |
|  word_count |     and      |   1   |  0.048449573172  |  None  |
|  word_count |     bags     |   1   |  0.165541436615  |  None  |
|  word_count |    wipes     |   1   | -0.0949937947268 |  None  |
+-------------+--------------+-------+------------------+--------+
[121713 rows x 5 columns]
Note: Only the head of the SFrame is

There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [15]:
print len(weights)
sperate_positive_weights = weights[weights['value'] >= 0]
sperate_negative_weights = weights[weights['value'] < 0]
num_positive_weights = len(sperate_positive_weights)
num_negative_weights = len(sperate_negative_weights)

print "Number of positive weights: %s " % num_positive_weights
print "Number of negative weights: %s " % num_negative_weights
print num_positive_weights + num_negative_weights

121713
Number of positive weights: 68419 
Number of negative weights: 53294 
121713


**Question:** How many weights are >= 0?

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [16]:
sample_test_data = test_data[10:13]
print sample_test_data['rating']
print sample_test_data

[5.0, 2.0, 1.0]
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|   Our Baby Girl Memory Book   | Absolutely love it and all... |  5.0   |
| Wall Decor Removable Decal... | Would not purchase again o... |  2.0   |
| New Style Trailing Cherry ... | Was so excited to get this... |  1.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+-----------+
|           word_count          | sentiment |
+-------------------------------+-----------+
| {'and': 2L, 'all': 1L, 'lo... |     1     |
| {'and': 1L, 'wall': 1L, 't... |     -1    |
| {'all': 1L, 'money': 1L, '... |     -1    |
+-------------------------------+-----------+
[3 rows x 5 columns]



Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [17]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [18]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using GraphLab Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [19]:
scores = sentiment_model.predict(sample_test_data, output_type='margin')
print scores

[6.734619727058284, -5.734130996759765, -14.668460404467742]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [20]:
for i in scores:
    if i >=0:
        print "1"
    else:
        print "-1"
            

1
-1
-1


Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from GraphLab Create.

In [21]:
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_data)

Class predictions according to GraphLab Create:
[1L, -1L, -1L]


**Checkpoint**: Make sure your class predictions match with the one obtained from GraphLab Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [22]:
import math

for x in scores:
    exp = math.exp( -x )
    prob = 1/ (1+exp)
    print prob

0.998812384838
0.0032232681818
4.26155799666e-07


**Checkpoint**: Make sure your probability predictions match the ones obtained from GraphLab Create.

In [23]:
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_data, output_type='probability')

Class predictions according to GraphLab Create:
[0.9988123848377186, 0.003223268181802263, 4.261557996658726e-07]


**Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [24]:
print test_data

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
| Baby Tracker&reg; - Daily ... | I love this journal and ou... |  4.0   |
| Nature's Lullabies First Y... | I love this little calende... |  5.0   |
| Nature's Lullabies Second ... | I had a hard time finding ... |  5.0   |
|  Lamaze Peekaboo, I Love You  | One of baby's first and fa... |  4.0   |
|  Lamaze Peekaboo, I Love You  | My son loved this book as ... |  5.0   |
|  Lamaze Peekaboo, I Love You  | Our baby loves this book &... |  5.0   |
| SoftPlay Giggle Jiggle Fun... | This bear is absolutely ad... |  2.0   |
| SoftPlay Peek-A-Boo Where'... | I bought two for recent ba... |  5.0   |
| Baby's First Year Undated ... | I searched high and low fo... |  5.0   |
+------------------------

In [25]:
test_data['probability'] = sentiment_model.predict(test_data, output_type='probability')
print test_data

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
| Baby Tracker&reg; - Daily ... | I love this journal and ou... |  4.0   |
| Nature's Lullabies First Y... | I love this little calende... |  5.0   |
| Nature's Lullabies Second ... | I had a hard time finding ... |  5.0   |
|  Lamaze Peekaboo, I Love You  | One of baby's first and fa... |  4.0   |
|  Lamaze Peekaboo, I Love You  | My son loved this book as ... |  5.0   |
|  Lamaze Peekaboo, I Love You  | Our baby loves this book &... |  5.0   |
| SoftPlay Giggle Jiggle Fun... | This bear is absolutely ad... |  2.0   |
| SoftPlay Peek-A-Boo Where'... | I bought two for recent ba... |  5.0   |
| Baby's First Year Undated ... | I searched high and low fo... |  5.0   |
+------------------------

# Top 20 rows that has positive reviews based on probabilities

In [26]:
#print test_data[-1]
twenty_rows = test_data.topk('probability',k=20)

len(twenty_rows)
for i in twenty_rows:
    print i['name'],'==',i['sentiment'],'==',i["probability"]


Britax Decathlon Convertible Car Seat, Tiffany == 1 == 1.0
Ameda Purely Yours Breast Pump - Carry All == 1 == 1.0
Traveling Toddler Car Seat Travel Accessory == -1 == 1.0
Shermag Glider Rocker Combo, Pecan with Oatmeal == 1 == 1.0
Cloud b Sound Machine Soother, Sleep Sheep == 1 == 1.0
JP Lizzy Chocolate Ice Classic Tote Set == 1 == 1.0
Fisher-Price Rainforest Melodies and Lights Deluxe Gym == 1 == 1.0
Lilly Gold Sit 'n' Stroll 5 in 1 Car Seat and Stroller Combination, Tuxedo Black (sunshade is not included in the offering) == 1 == 1.0
Fisher-Price Deluxe Jumperoo == 1 == 1.0
North States Supergate Pressure Mount Clear Choice Wood Gate == 1 == 1.0
Munchkin Mozart Magic Cube == 1 == 1.0
Britax Marathon Convertible Car Seat, Granite == 1 == 1.0
Wizard Convertible Car Seat with LATCH in Midnight Print == 1 == 1.0
Capri Stroller - Red Tech == 1 == 1.0
Peg Perego Primo Viaggio Car Seat / Infant Carrier with LATCH Base - Black Sable == 1 == 1.0
HALO SleepSack Micro-Fleece Wearable Blanket, So

In [27]:
print twenty_rows['review'][0]

I researched a few different seats to put in our new Odyssey van in September (07). Our daughter just turned two and we wanted to get a new car seat for her so her Evenflo Triumph could go in my husbands car. She had been using the Triumph since she was about 9 months (rear-facing), and then forward facing from about 14 months. We really loved the Triumph, but I wanted something she could grow into. Meaning - getting the most out of a car seat height and weight wise for the longest period of time.Britax was one of the only ones I found, that actually makes a heavy-duty quality seat that I knew would last. Sure, I looked at a few others that ranked up there with height and weight (i.e. Eddie Bauer, Cosco, and Graco), but none of these compared in quality and durability! We made the trip to Babies r Us so we could "test" the few different seats I had chosen from Consumer Reports ratings. The Eddie Bauer deluxe booster was out because the first time I snapped the metal part into the crotc

**Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

# Top 20 rows that has negative reviews based on probabilities

In [28]:
#print test_data[-1]
twenty_rows = test_data.topk('probability',k=20,reverse = True)

len(twenty_rows)
for i in twenty_rows:
    print i['name'],'==',i['sentiment'],'==',i["probability"]

Jolly Jumper Arctic Sneak A Peek Infant Car Seat Cover Black == 1 == 7.80415068215e-100
Levana Safe N'See Digital Video Baby Monitor with Talk-to-Baby Intercom and Lullaby Control (LV-TW501) == -1 == 6.83650885517e-25
Snuza Portable Baby Movement Monitor == -1 == 2.12654510825e-24
Fisher-Price Ocean Wonders Aquarium Bouncer == -1 == 2.2458208078e-23
VTech Communications Safe &amp; Sounds Full Color Video and Audio Monitor == -1 == 1.32962966147e-22
Safety 1st High-Def Digital Monitor == -1 == 2.0687209747e-20
Chicco Cortina KeyFit 30 Travel System in Adventure == -1 == 5.93881994674e-20
Prince Lionheart Warmies Wipes Warmer == -1 == 6.28510016543e-20
Valco Baby Tri-mode Twin Stroller EX- Hot Chocolate == -1 == 8.05528712696e-20
Adiri BPA Free Natural Nurser Ultimate Bottle Stage 1 White, Slow Flow (0-3 months) == -1 == 8.4652172495e-20
Munchkin Nursery Projector and Sound System, White == -1 == 1.52853945171e-19
The First Years True Choice P400 Premium Digital Monitor, 2 Parent Unit ==

In [29]:
print twenty_rows['review'][0]

I am a "research-aholic" in general and have researched many different baby products over the past 5 years due to being a mother of 3 and wanting to get the best for the price and not wanting to settle for less...  I live in the Northeast US, so it gets really cold and the weather changes frequently and in extremes on a daily, if not hourly basis.  Anyway, here are the brands I've used/owed and/or researched extensively and the comparisons:Jolly Jumper Arctic Sneak A Peek Infant Car Seat Cover (JJA)Jolly Jumper Sneak A Peek Infant Car Seat Cover (JJ)Summer Infant Poshpouch (SP)Cozy Car Seat Microfiber and Fleece Cover (CC)Baby's Cozy World Microfiber & Fleece Carrier Cover (BC) (yes the CC and the BC are different many people do not know this....)JJ Cole infant/original bundle me (JJC)JJ Cole toddler bundle me  (JJT)OK...Jolly Jumper Artic (JJA) is awesome, I would rate it the best one out of all.  It is a "shower cap" cover for the infant car seat, it has an attached blanket inside an

**Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [30]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    ## YOUR CODE HERE
    
    model_predictions = model.predict(data)
    
    
    
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    num_correct = 0
    for i in range(len(data)):
        if true_labels[i] == model_predictions[i]:
            num_correct = num_correct + 1
    
    

    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    
    accuracy = num_correct / len(data)
    
    
    return accuracy

In [31]:
test_data['model_prediction'] = sentiment_model.predict(test_data)
print test_data

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
| Baby Tracker&reg; - Daily ... | I love this journal and ou... |  4.0   |
| Nature's Lullabies First Y... | I love this little calende... |  5.0   |
| Nature's Lullabies Second ... | I had a hard time finding ... |  5.0   |
|  Lamaze Peekaboo, I Love You  | One of baby's first and fa... |  4.0   |
|  Lamaze Peekaboo, I Love You  | My son loved this book as ... |  5.0   |
|  Lamaze Peekaboo, I Love You  | Our baby loves this book &... |  5.0   |
| SoftPlay Giggle Jiggle Fun... | This bear is absolutely ad... |  2.0   |
| SoftPlay Peek-A-Boo Where'... | I bought two for recent ba... |  5.0   |
| Baby's First Year Undated ... | I searched high and low fo... |  5.0   |
+------------------------

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [32]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.9145368370530358

In [33]:
sentiment_model.evaluate(test_data)

{'accuracy': 0.9145368370530358,
 'auc': 0.9250003098558346,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      -1      |        -1       |  3798 |
 |      -1      |        1        |  1443 |
 |      1       |        -1       |  1406 |
 |      1       |        1        | 26689 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9493303928717519,
 'log_loss': 0.3707618266205636,
 'precision': 0.9487060998151571,
 'recall': 0.9499555080975263,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+----------------+----------------+-------+------+
 | threshold |      fpr       |      tpr       |   p   |  n   |
 +-----------+----------------+----------------+-------+------+
 |    0.0    |      1.0       | 

**Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [34]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [35]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [36]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

Let's see what the first example of the dataset looks like:

In [37]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

In [38]:
print train_data[0]['word_count']

{'and': 3L, 'love': 1L, 'it': 3L, 'highly': 1L, 'osocozy': 1L, 'bags': 1L, 'leak': 1L, 'moist': 1L, 'does': 1L, 'recommend': 1L, 'was': 1L, 'wipes': 1L, 'disappointed': 1L, 'early': 1L, 'not': 2L, 'now': 1L, 'holder': 1L, 'wipe': 1L, 'keps': 1L, 'wise': 1L, 'i': 1L, 'planet': 1L, 'my': 2L, 'came': 1L}


Since we are only working with a subset of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [39]:
print train_data[0]['word_count_subset']

{'love': 1L, 'disappointed': 1L}


## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [40]:
simple_model = graphlab.logistic_classifier.create(train_data,
                                                   target = 'sentiment',
                                                   features=['word_count_subset'],
                                                   validation_set=None)
simple_model

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 6        | 0.847620     | 0.866815          |

| 6         | 7        | 0.978545     | 0.866815          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 21
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 20

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 6
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 1.012

Settings
--------
Log-likelihood                 : 44323.7254

Highest Positive Coefficients
-----------------------------
word_count_subset[loves]       : 1.6773
word_count_subset[perfect]     : 1.5145
word_count_subset[love]        : 1.3654
(intercept)                    : 1.2995
word_count_subset[easy]        : 1.1937

Lowest Negative Coefficients
----------------------------
word_count_subset[disappointed] : -2.3551
wor

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [41]:
simple_model.evaluate(test_data)

{'accuracy': 0.8693004559635229,
 'auc': 0.828809641857926,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        -1       |  469  |
 |      -1      |        -1       |  1353 |
 |      -1      |        1        |  3888 |
 |      1       |        1        | 27626 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9269070106863058,
 'log_loss': 0.3296871005223945,
 'precision': 0.8766262613441645,
 'recall': 0.983306638191849,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+----------------+----------------+-------+------+
 | threshold |      fpr       |      tpr       |   p   |  n   |
 +-----------+----------------+----------------+-------+------+
 |    0.0    |      1.0       |   

In [42]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8693004559635229

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [43]:
print simple_model.coefficients

+-------------------+--------------+-------+-----------------+-----------------+
|        name       |    index     | class |      value      |      stderr     |
+-------------------+--------------+-------+-----------------+-----------------+
|    (intercept)    |     None     |   1   |   1.2995449552  | 0.0120888541331 |
| word_count_subset | disappointed |   1   |  -2.35509250061 | 0.0504149888557 |
| word_count_subset |     love     |   1   |  1.36543549368  | 0.0303546295109 |
| word_count_subset |     well     |   1   |  0.504256746398 |  0.021381300631 |
| word_count_subset |   product    |   1   | -0.320555492996 | 0.0154311321362 |
| word_count_subset |    loves     |   1   |  1.67727145556  | 0.0482328275384 |
| word_count_subset |    little    |   1   |  0.520628636025 | 0.0214691475665 |
| word_count_subset |     work     |   1   | -0.621700012425 | 0.0230330597946 |
| word_count_subset |     easy     |   1   |  1.19366189833  |  0.029288869202 |
| word_count_subset |    gre

Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [44]:
simple_model_weights = simple_model.coefficients.sort('value', ascending=False)

**Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [45]:
count = 0
simple_model_positive_words = []

for i in range(len(simple_model_weights)):
    if simple_model_weights['value'][i] >= 0 and simple_model_weights['name'][i] != '(intercept)':
        #print simple_model_weights['value'][i], simple_model_weights['name'][i], simple_model_weights['class'][i], simple_model_weights['index'][i]
        simple_model_positive_words.append(simple_model_weights['index'][i])
        count = count + 1
        
print count

10


**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [50]:
sentiment_model_weights = sentiment_model.coefficients.sort('value', ascending=False)
sentiment_model_weights['positive_negative_weights'] = sentiment_model_weights['value'].apply(lambda x: 'positive' if x>=0 else 'negative')
print sentiment_model_weights.groupby(key_columns='positive_negative_weights',
                         operations={'count': graphlab.aggregate.COUNT()})
positive_name = sentiment_model_weights["index"][sentiment_model_weights['positive_negative_weights'] == 'positive']
negative_name = sentiment_model_weights['index'][sentiment_model_weights['positive_negative_weights'] == 'negative']
print 'lenght of sentiment_model_weights: ',len(sentiment_model_weights)
print 'lenght of positive_name: ',len(positive_name)
print 'lenght of negative_name: ',len(negative_name)
print 'lenght of positive_name plus lenght of negative_name: ',len(positive_name) + len(negative_name)


+---------------------------+-------+
| positive_negative_weights | count |
+---------------------------+-------+
|          positive         | 68419 |
|          negative         | 53294 |
+---------------------------+-------+
[2 rows x 2 columns]

lenght of sentiment_model_weights:  121713
lenght of positive_name:  68419
lenght of negative_name:  53294
lenght of positive_name plus lenght of negative_name:  121713


# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [52]:
get_classification_accuracy(sentiment_model,train_data,train_data['sentiment'])

0.979440247046831

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [53]:
get_classification_accuracy(simple_model,train_data,train_data['sentiment'])

0.8668150746537147

**Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [54]:
get_classification_accuracy(sentiment_model,test_data,test_data['sentiment'])

0.9145368370530358

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [55]:
get_classification_accuracy(simple_model,test_data,test_data['sentiment'])

0.8693004559635229

**Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [56]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print num_positive
print num_negative

112164
21252


Now compute the accuracy of the majority class classifier on **test_data**.

**Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [57]:
print (test_data['sentiment'] == +1).sum()/len(test_data['sentiment'])

0.842782577394


**Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?